# Gradient vs OLS

Vamos implementar a regressão linear por gradiente descendente e por OLS para comparar os resultado

## Importações

In [1]:
import numpy as np
import plotly.graph_objects as go

## Gerando os dados que usaremos para gerar uma linha

In [2]:
x_test_1 = np.array([1, 5])
y_test_1 = np.array([2, 3])

## Código para geração de gráficos para teste

In [3]:
from typing import Callable
def plot_line_with_data_points(line_function: Callable[[np.ndarray], np.ndarray], x: np.ndarray, y: np.ndarray, title: str):
    x_line = np.linspace(np.min(x) - 1, np.max(x) + 1, 1000)
    y_line = line_function(x_line)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x_line, y=y_line, mode='lines', name='Line'))
    fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name='Data Points'))
    fig.update_layout(title=title)
    fig.show()

plot_line_with_data_points(lambda x: x, x_test_1, y_test_1, 'teste')

## Gradiente descendente

In [4]:
def compute_cost(x, y, w, b):
   
    m = x.shape[0] 
    cost = 0
    
    for i in range(m):
        f_wb = w * x[i] + b
        cost = cost + (f_wb - y[i])**2
    total_cost = 1 / (2 * m) * cost

    return total_cost

def compute_gradient(x, y, w, b): 
    """
    Computes the gradient for linear regression 
    Args:
      x (ndarray (m,)): Data, m examples 
      y (ndarray (m,)): target values
      w,b (scalar)    : model parameters  
    Returns
      dj_dw (scalar): The gradient of the cost w.r.t. the parameters w
      dj_db (scalar): The gradient of the cost w.r.t. the parameter b     
     """
    
    # Number of training examples
    m = x.shape[0]    
    dj_dw = 0
    dj_db = 0
    
    for i in range(m):  
        f_wb = w * x[i] + b 
        dj_dw_i = (f_wb - y[i]) * x[i] 
        dj_db_i = f_wb - y[i] 
        dj_db += dj_db_i
        dj_dw += dj_dw_i 
    dj_dw = dj_dw / m 
    dj_db = dj_db / m 
        
    return dj_dw, dj_db

def gradient_descent(x, y,alpha, num_iters, cost_function, gradient_function, print_each: int): 
    b = 0
    w = 0
    
    for i in range(num_iters):
        # Calculate the gradient and update the parameters using gradient_function
        dj_dw, dj_db = gradient_function(x, y, w , b)     

        # Update Parameters using equation (3) above
        b = b - alpha * dj_db                            
        w = w - alpha * dj_dw                            

        # Print cost every at intervals 10 times or as many iterations if < 10
        if (i + 1) % print_each == 0:
            cost = cost_function(x, y, w, b)
            plot_line_with_data_points(lambda x: w * x + b, x, y, f'Iteration {i + 1}: Cost = {cost}, w = {w}, b = {b}')
            
 
    return w, b


# Test the gradient descent function
alpha = 0.1
num_iters = 200
print_each = 20
w, b = gradient_descent(x_test_1, y_test_1, alpha, num_iters, compute_cost, compute_gradient, print_each)

## OLS

In [14]:
def calculate_params_by_OLS(x: np.ndarray, y: np.ndarray):
    """
    Calculate the parameters w and b using the Ordinary Least Squares method
    Args:
      x (ndarray (m,)): Data, m examples 
      y (ndarray (m,)): target values
    Returns:
      w, b (scalar): model parameters
    """
    x = np.concatenate((x, np.ones((x.shape[0], 1))), axis=1)
    x_transpose = x.T

    w = np.linalg.inv(x_transpose.dot(x)).dot(x_transpose).dot(y)
    
    return w

w = calculate_params_by_OLS(x_test_1.reshape(-1, 1), y_test_1.reshape(-1, 1))

print(w)

[[0.25]
 [1.75]]


In [29]:
def calculate_params_ridge(x: np.ndarray, y: np.ndarray, _lambda: float):
    """
    Calculate the parameters w and b using the Ridge Regression method
    Args:
      x (ndarray (m,)): Data, m examples 
      y (ndarray (m,)): target values
      _lambda (scalar): Regularization parameter
    Returns:
      w, b (scalar): model parameters
    """
    x = np.concatenate((x, np.ones((x.shape[0], 1))), axis=1)
    x_transpose = x.T

    w = np.linalg.inv(x_transpose.dot(x) + (_lambda * np.identity(x.shape[1]))).dot(x_transpose).dot(y)
    
    return w

w = calculate_params_ridge(x_test_1.reshape(-1, 1), y_test_1.reshape(-1, 1), 0.000001)
w

array([[0.25000062],
       [1.74999725]])